#Utils

In [1]:
import itertools
import copy
import random
import json
import numpy as np

In [2]:
def pprint(obj):
    '''
        Indented print of an object
    '''
    print(json.dumps(obj, indent=3))

#Formula
Formula definition.
Look at the example after the next cell to understand how to instantiate a formula.

In [3]:
class Var:
    '''
        A var is a name, if it is negated and its value (True or False)
        e.g. var: "x", "negate = True", value = "False" -> ¬x, x = False -> True
    '''
    def __init__(self, name, negate = False, value = None):
        self.name = name
        self.negate = negate
        self.value = value
    
    def set_value(self, value):
        self.value = value if not self.negate else not value

    def __str__(self):
        return self.name if not self.negate else "¬" + self.name

    def __repr__(self):
        return self.name if not self.negate else "¬" + self.name
    
    def __deepcopy__(self, memo = None, _nil = []):
        return Var(self.name, self.negate, self.value)
    
class Clause:
    '''
        A clause is a list of conjunctions of vars
        e.g. (x ∨ y ∨ z ∨ ¬w)
    '''
    def __init__(self, list_var):
        self.list_var = list_var
    
    def evaluate(self, interpretation, f):
        '''
            Evaluate if the clause is True with a full interpretation
        '''
        for var in self.list_var:
            i = f.var2id[var.name]
            var.set_value(interpretation[i])
        out = False
        for var in self.list_var:
            out = out or var.value
        return out

    def evaluate_partial(self, interpretation, f):
        '''
            Evaluate if the clause is True with a partial interpretation
        '''
        for var in self.list_var:
            if interpretation.get(var.name) is not None:
                var.set_value(interpretation[var.name])
        
        is_true = False
        no_assign = False
        for var in self.list_var:
            if var.value is not None:
                if var.value:
                    return True
            else:
                no_assign = True
        if no_assign:
            return 42
        return False

    def __str__(self):
        out = "("
        for var in self.list_var:
            out += str(var) + " ∨ "
        out = out[:-3] + ")"
        return out

    def __repr__(self):
        out = "("
        for var in self.list_var:
            out += str(var) + " ∨ "
        out = out[:-3] + ")"
        return out

    def __deepcopy__(self, memo = None, _nil = []):
        list_var = copy.deepcopy(self.list_var, memo = None, _nil = [])
        return Clause(list_var)

class Formula:
    '''
        A formula is a list of disjunctions of clauses
        e.g. (r ∨ x ∨ ¬y ∨ z ∨ ¬w) ∧ (a ∨ ¬x ∨ y ∨ ¬w) ∧ (b ∨ ¬a ∨ w)
        This class also contains some useful information:
            - number of different variables
            - a mapping from an id to the variable and vice versa
    '''
    def __init__(self, list_clause):
        self.list_clause = list_clause
        self.num_vars, self.id2var, self.var2id = self.__vars()

    def evaluate(self, i):
        '''
            Evaluate if the formula is True with a full interpretation
        '''
        c_true = 0
        for c in self.list_clause:
            if c.evaluate(i, self):
                c_true += 1
        return len(self.list_clause) == c_true, c_true

    def evaluate_partial(self, i):
        '''
            Evaluate if the formula is True with a partial interpretation
        '''
        for c in self.list_clause:
            if c.evaluate_partial(i, self) == False:
                return False
            elif c.evaluate_partial(i, self) == 42:
                return 42
        return True

    def partial_interpretation(self, i):
        '''
            Apply partial interpretation to a formula, removing vars and/or clauses if necessary
        '''
        for var, truth in i.items():
            to_remove = []
            for clause in self.list_clause:
                if clause.evaluate_partial(i, self) == True:
                    to_remove.append(clause)
                else:
                    to_remove_var = []
                    for var_c in clause.list_var:
                        if var_c.name == var:
                            if var_c.negate != truth:
                                to_remove.append(clause)
                                break
                            else:
                                to_remove_var.append(var_c)
                    for v in to_remove_var:
                        if len(clause.list_var) == 1:
                            to_remove.append(clause)
                        else:
                            clause.list_var.remove(v)
            for c in to_remove:
                self.list_clause.remove(c)

    def __vars(self):
        var_set = set()
        for clause in self.list_clause:
            for var in clause.list_var:
                var_set.add(var.name)
        id2var = {}
        var2id = {}
        for i, var in enumerate(var_set):
            id2var[i] = var
            var2id[var] = i
        return len(var_set), id2var, var2id

    def __str__(self):
        out = ""
        for c in self.list_clause:
            out += str(c) + " ∧ "
        out = out[:-3]
        return out if len(out) > 0 else "( )"

    def __repr__(self):
        out = ""
        for c in self.list_clause:
            out += str(c) + " ∧ "
        out = out[:-3]
        return out if len(out) > 0 else "( )"

    def __deepcopy__(self, memo = None, _nil = []):    
        list_clause = copy.deepcopy(self.list_clause)
        return Formula(list_clause)

In [4]:
# Var = name, negation. Remember that negation = True means that the variable is negated
vars = [] 
vars.append([Var("r", False), Var("x", False), Var("y", True), Var("z", False), Var("w", True)])
vars.append([Var("a", False), Var("x", True), Var("y", False), Var("w", True)])
vars.append([Var("b", False), Var("a", True), Var("w", False)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)
print(f)

(r ∨ x ∨ ¬y ∨ z ∨ ¬w) ∧ (a ∨ ¬x ∨ y ∨ ¬w) ∧ (b ∨ ¬a ∨ w)


#GSAT
First sat solver.

Gsat starts from an initial interpretation and changes the value of the variable in the interpretation that maximizes True clauses. Can fall into local maxima.

This is a modified version that also tells us whether it can find a solution in at least $max\_hop$ steps

In [5]:
def gsat(T, init_I, max_flips = 1000, max_hop = 100000, max_try = 1000):
    '''
        params:
            T: the formula whose satisfiability must be verified
            init_I: initial interpretation
            max_flips: maximum number of variable value changes in the interpretation after which it stops searching along that branch
            max_hop: check a solution that uses at least this number of steps
            max_try: after this number of steps, it stops the search and states that the solution has not been found
    '''
    num_var = T.num_vars
    name_max = 0
    interpretations = [[init_I, 0, (name_max, name_max)]]
    interpretations_list = []
    found = False
    current_try = 0
    while interpretations != []:
        current_try += 1
        if current_try >= max_try:
            return {"sol_found": False, "sol_found_with_more_hop": True, "interpretations": []}
        interpretation = interpretations.pop(0)
        interpretations_list.append(interpretation)
        if T.evaluate(interpretation[0])[0] and interpretation[1] >= max_hop:
            return {"sol_found": True, "sol_found_with_more_hop": True, "interpretations": interpretations_list}
        elif T.evaluate(interpretation[0])[0] and interpretation[1] < max_hop:
            found = True
            continue 
        if interpretation[1] >= max_flips:
            continue
        max = 0
        max_interpretation = []
        for i in range(num_var):
            interpretation_flip = copy.deepcopy(interpretation)
            interpretation_flip[0][i] = not interpretation_flip[0][i]
            eval_interpretation = T.evaluate(interpretation_flip[0])[1]
            if eval_interpretation > max:
                max = eval_interpretation
                max_interpretation = []
                max_interpretation.append(interpretation_flip)
            elif eval_interpretation == max:
                max_interpretation.append(interpretation_flip)
        for i in range(len(max_interpretation)):
            max_interpretation[i][1] += 1
            name_max += 1
            max_interpretation[i][2] = (interpretation[2][1], name_max)
            interpretations.append(max_interpretation[i])
    return {"sol_found": True, "sol_found_with_more_hop": False, "interpretations": interpretations_list}

def print_result(sol, f, init_I):
    '''
        Print the path to the solution found in gsat, or print "no solution"
    '''
    print("Formula:", f)
    print("Initial Interpretation:", [(f.id2var[i], var_i) for i, var_i in enumerate(init_I)], "\n")
    if not sol["sol_found"]:
        print("No solution found")
    if sol["sol_found"]:
        print("Solution Found")
        if sol["sol_found_with_more_hop"]:
            print("In more hop than the input")
        else:
            print("In less hop than the input")
        i_win_dict = {}
        for interp in sol["interpretations"]:
            i_win_dict[interp[-1][1]] = interp[:-1] + [interp[-1][0]]

        current_key = max(list(i_win_dict.keys()))
        path = []
        while True:
            current_value = i_win_dict[current_key]
            path.append(current_value[0])
            current_key = current_value[-1]
            if current_key == 0:
                break

        path.append(i_win_dict[current_key][0])
        path.reverse()
        path2print = [("start", [(f.id2var[i], p) for i, p in enumerate(path[0])])]
        for i in range(1, len(path), 1):
            from_a = path[i-1]
            to_b = path[i]
            for j, var in enumerate(from_a):
                if var != to_b[j]:
                    difference = str(f.id2var[j]) + ": " + str(var) + " -> " + str(to_b[j])
                    break
            path2print.append((difference, [(f.id2var[i], p) for i, p in enumerate(path[i])]))
        print()
        print("Path")
        for i, p in enumerate(path2print):
            print(i, p)

In [6]:
# Example, no solution formula: (x) ∧ (¬x)
print("EXAMPLE 1")
vars = []
vars.append([Var("x", False)])
vars.append([Var("x", True)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)
sol = gsat(f, [True])
print_result(sol, f, [True])
print()

# Example, solution with less than hop passed (r ∨ x ∨ ¬y ∨ z ∨ ¬w) ∧ (a ∨ ¬x ∨ y ∨ ¬w) ∧ (b ∨ ¬a ∨ w)
print("EXAMPLE 2")
vars = []
vars.append([Var("r", False), Var("x", False), Var("y", True), Var("z", False), Var("w", True)])
vars.append([Var("a", False), Var("x", True), Var("y", False), Var("w", True)])
vars.append([Var("b", False), Var("a", True), Var("w", False)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)
for i in range(10000):
    init_I = [random.choice([True, False]) for i in range(f.num_vars)]
    sol = gsat(f, init_I) 
    if len(sol["interpretations"]) > 1:
        break
print_result(sol, f, init_I)
print()

# Example, solution with more than hop passed
# ((x ∨ y ∨ z ∨ w) ∧ (x ∨ y ∨ z ∨ ¬w) ∧ (x ∨ y ∨ ¬z ∨ w) ∧ (¬x ∨ ¬y ∨ z ∨ w) ∧ (x ∨ ¬y ∨ z ∨ w) ∧ (¬x ∨ y ∨ ¬z ∨ w) ∧ (¬x ∨ y ∨ z ∨ ¬w) ∧ (¬x ∨ y ∨ z ∨ w)
print("EXAMPLE 3")
vars = []
vars.append([Var("x", False), Var("y", False), Var("z", False), Var("w", False)])
vars.append([Var("x", False), Var("y", False), Var("z", False), Var("w", True)])
vars.append([Var("x", False), Var("y", False), Var("z", True), Var("w", False)])
vars.append([Var("x", True), Var("y", True), Var("z", False), Var("w", False)])
vars.append([Var("x", False), Var("y", True), Var("z", False), Var("w", False)])
vars.append([Var("x", True), Var("y", False), Var("z", True), Var("w", False)])
vars.append([Var("x", True), Var("y", False), Var("z", False), Var("w", True)])
vars.append([Var("x", True), Var("y", False), Var("z", False), Var("w", False)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)
sol = gsat(f, [False]*f.num_vars, max_hop = 7)

print_result(sol, f, [False]*f.num_vars)

EXAMPLE 1
Formula: (x) ∧ (¬x)
Initial Interpretation: [('x', True)] 

No solution found

EXAMPLE 2
Formula: (r ∨ x ∨ ¬y ∨ z ∨ ¬w) ∧ (a ∨ ¬x ∨ y ∨ ¬w) ∧ (b ∨ ¬a ∨ w)
Initial Interpretation: [('a', True), ('w', False), ('r', False), ('z', False), ('b', False), ('y', False), ('x', True)] 

Solution Found
In less hop than the input

Path
0 ('start', [('a', True), ('w', False), ('r', False), ('z', False), ('b', False), ('y', False), ('x', True)])
1 ('b: False -> True', [('a', True), ('w', False), ('r', False), ('z', False), ('b', True), ('y', False), ('x', True)])

EXAMPLE 3
Formula: (x ∨ y ∨ z ∨ w) ∧ (x ∨ y ∨ z ∨ ¬w) ∧ (x ∨ y ∨ ¬z ∨ w) ∧ (¬x ∨ ¬y ∨ z ∨ w) ∧ (x ∨ ¬y ∨ z ∨ w) ∧ (¬x ∨ y ∨ ¬z ∨ w) ∧ (¬x ∨ y ∨ z ∨ ¬w) ∧ (¬x ∨ y ∨ z ∨ w)
Initial Interpretation: [('z', False), ('w', False), ('y', False), ('x', False)] 

Solution Found
In more hop than the input

Path
0 ('start', [('z', False), ('w', False), ('y', False), ('x', False)])
1 ('x: False -> True', [('z', False), ('w', False), ('y', Fal

#Unit Propagation
Formula simplification.

If a clause has only one literal, then that literal has only one possible assignment, and after the assignment all other clauses must be modified.

In [7]:
def UP(f, ind = 0, ind_char = "  "):
    '''
        ind and ind_char are used to indent the exploration branches in DPLL and backtracking algorithms
    '''
    I_new = {}
    while(find_single_lit(f)[0]):
        single_lit = find_single_lit(f)[1]
        I_new[single_lit.name] = not single_lit.negate
        print(("|" + ind_char) * ind + str(f), "-", "Single literal:",single_lit)
        to_remove = []
        for clause in f.list_clause:
            delete_clause = False
            for var in clause.list_var:
                if var.name == single_lit.name:
                    if var.negate == single_lit.negate:
                        delete_clause = True
                    else:
                        if len(clause.list_var) == 1:
                            to_remove.append(clause)
                        else:
                            clause.list_var.remove(var)
                    break
            if delete_clause:
                to_remove.append(clause)
        for c in to_remove:
            f.list_clause.remove(c)
    # print(f)
    return f, I_new

def find_single_lit(f):
    '''
        Search a unit clause
    '''
    for clause in f.list_clause:
        if len(clause.list_var) == 1:
            return True, clause.list_var[0]
    return False, None

In [8]:
vars = []
vars.append([Var("x1", False), Var("x2", True), Var("x3", True)])
vars.append([Var("x1", True), Var("x2", False), Var("x3", True)])
vars.append([Var("x4", False), Var("x5", True)])
vars.append([Var("x5", False)])
vars.append([Var("x1", True), Var("x3", False), Var("x4", True)])
vars.append([Var("x2", True), Var("x4", True)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)
UP(f)

(x1 ∨ ¬x2 ∨ ¬x3) ∧ (¬x1 ∨ x2 ∨ ¬x3) ∧ (x4 ∨ ¬x5) ∧ (x5) ∧ (¬x1 ∨ x3 ∨ ¬x4) ∧ (¬x2 ∨ ¬x4) - Single literal: x5
(x1 ∨ ¬x2 ∨ ¬x3) ∧ (¬x1 ∨ x2 ∨ ¬x3) ∧ (x4) ∧ (¬x1 ∨ x3 ∨ ¬x4) ∧ (¬x2 ∨ ¬x4) - Single literal: x4
(x1 ∨ ¬x2 ∨ ¬x3) ∧ (¬x1 ∨ x2 ∨ ¬x3) ∧ (¬x1 ∨ x3) ∧ (¬x2) - Single literal: ¬x2


((¬x1 ∨ ¬x3) ∧ (¬x1 ∨ x3), {'x5': True, 'x4': True, 'x2': False})

#Pure Literal
Formula simplification.

If a literal appears only negated or only not negated in the clauses then there is only one way to assign this literal which leads to the truth of the clause and thus its elimination.

In [9]:
def pure_literal(f, ind = 0, ind_char = "  "):
    '''
        ind and ind_char are used to indent the exploration branches in DPLL and backtracking algorithms
    '''
    I_new = {}
    cont = 0
    while True:
        matrix = np.zeros((f.num_vars, len(f.list_clause)))
        for i in range(f.num_vars):
            for j in range(len(f.list_clause)):
                clause_j = f.list_clause[j]
                is_present = False
                for var_j in clause_j.list_var:
                    if f.id2var[i] == var_j.name:
                        is_present = True
                        break
                if is_present:
                    if var_j.negate:
                        matrix[i][j] -= 1
                    else:
                        matrix[i][j] += 1
        pure_literals = []
        have_pl = False
        no_pl = 0
        for i, var in enumerate(matrix):
            if (all(var<=0) or all(var>=0)) and not all(var == 0):
                pure_literals.append(f.id2var[i])
                have_pl = True
            else:
                no_pl += 1
        if not have_pl or matrix.shape[1] == 0 or no_pl == matrix.shape[1]:
            break
        print(("|" + ind_char) * ind + "NEW STEP")
        for pure_literal in pure_literals:
            to_remove = []
            var_pl = None
            for clause in f.list_clause:
                for var in clause.list_var:
                    if var.name == pure_literal:
                        to_remove.append(clause)
                        var_pl = var
                        break
            if var_pl is not None:
                I_new[var_pl.name] = not var_pl.negate
                print(("|" + ind_char) * ind + str(f), "-", "Pure li teral:", var_pl)
            for c in to_remove:
                f.list_clause.remove(c)
    return f, I_new

In [10]:
vars = []
vars.append([Var("a", False), Var("b", True), Var("c", True)])
vars.append([Var("a", False), Var("c", False)])
vars.append([Var("b", False), Var("d", True)])
vars.append([Var("b", False), Var("c", True)])
vars.append([Var("b", True), Var("c", False)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)
pure_literal(f)

NEW STEP
(a ∨ ¬b ∨ ¬c) ∧ (a ∨ c) ∧ (b ∨ ¬d) ∧ (b ∨ ¬c) ∧ (¬b ∨ c) - Pure li teral: a
(b ∨ ¬d) ∧ (b ∨ ¬c) ∧ (¬b ∨ c) - Pure li teral: ¬d


((b ∨ ¬c) ∧ (¬b ∨ c), {'a': True, 'd': False})

#Backtracking
Second sat solver.

The backtracking algorithm is a recursive algorithm that given a partial interpretation, randomly chooses a variable not present in the interpretation and runs the algorithm on a new partial interpretation that contains this variable once set to True and once set to True.

The random choice of variables could lead to very long executions when a systematic choice would have shut it down immediately

In [11]:
def backtracking(f, I, f_prime = None, ind = 1, ind_char = "    "):
    '''
        f: the formula to check
        I: the partial interpretation
        f_prime: a copy of the original formula during recursion for verification on partial interpretation
        ind, ind_char: for the branch indentation
    '''
    f_check = copy.deepcopy(f) if f_prime is None else f_prime
    print(("|" + ind_char) * (ind-1) + "START BRANCH")
    if f_check.evaluate_partial(I) == True:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, I
    if f_check.evaluate_partial(I) == False:
        print(("|" + ind_char) * (ind-1) + "NO SOLUTION FOUND!!!")
        return False, None
    
    if len(f.list_clause) == 0:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, I
    for var in f.var2id.keys():
        if I.get(var) is None:
            xi = var
            break
    I_new_true = copy.deepcopy(I)
    I_new_false = copy.deepcopy(I)
    I_new_true[xi] = True
    I_new_false[xi] = False

    print(("|" + ind_char) * ind + "PARTIAL INTERPRETATION")
    f.partial_interpretation(I)
    
    if f.evaluate_partial(I) == False:
        print(("|" + ind_char) * (ind-1) + "NO SOLUTION FOUND!!!")
        print(("|" + ind_char) * (ind-1))
        return False, None
    if f.evaluate_partial(I) == True:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, I
    if len(f.list_clause) == 0:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, I
    print(("|" + ind_char) * ind + str(f))
    print(("|" + ind_char) * ind + str(I))
    print(("|" + ind_char) * ind)

    print(("|" + ind_char) * ind + "RANDOM VARIABLE RECURSION -", xi, "= True")
    out_b_true = backtracking(copy.deepcopy(f), I_new_true, f_prime = f_check, ind = ind + 1, ind_char = ind_char)
    if out_b_true[0]:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, out_b_true[1]
    print(("|" + ind_char) * ind + "RANDOM VARIABLE RECURSION -", xi, "= False")
    out_b_false = backtracking(copy.deepcopy(f), I_new_false, f_prime = f_check, ind = ind + 1, ind_char = ind_char)
    if out_b_false[0]:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, out_b_false[1]
    print(("|" + ind_char) * (ind-1) + "NO SOLUTION FOUND!!!")
    print(("|" + ind_char) * (ind-1))
    return False, None

In [12]:
vars = []
vars.append([Var("x4", False)])
vars.append([Var("x4", True), Var("x1", False)])
vars.append([Var("x1", True), Var("x2", False)])
vars.append([Var("x2", True), Var("x3", False)])
vars.append([Var("x3", True), Var("x5", False)])
vars.append([Var("x5", True)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)
result =backtracking(f, {})
print("RESULT")
print(result)

START BRANCH
|    PARTIAL INTERPRETATION
|    (x4) ∧ (¬x4 ∨ x1) ∧ (¬x1 ∨ x2) ∧ (¬x2 ∨ x3) ∧ (¬x3 ∨ x5) ∧ (¬x5)
|    {}
|    
|    RANDOM VARIABLE RECURSION - x5 = True
|    START BRANCH
|    |    PARTIAL INTERPRETATION
|    |    (x4) ∧ (¬x4 ∨ x1) ∧ (¬x1 ∨ x2) ∧ (¬x2 ∨ x3)
|    |    {'x5': True}
|    |    
|    |    RANDOM VARIABLE RECURSION - x4 = True
|    |    START BRANCH
|    |    |    PARTIAL INTERPRETATION
|    |    |    (x1) ∧ (¬x1 ∨ x2) ∧ (¬x2 ∨ x3)
|    |    |    {'x5': True, 'x4': True}
|    |    |    
|    |    |    RANDOM VARIABLE RECURSION - x2 = True
|    |    |    START BRANCH
|    |    |    |    PARTIAL INTERPRETATION
|    |    |    |    (x1) ∧ (x3)
|    |    |    |    {'x5': True, 'x4': True, 'x2': True}
|    |    |    |    
|    |    |    |    RANDOM VARIABLE RECURSION - x1 = True
|    |    |    |    START BRANCH
|    |    |    |    |    PARTIAL INTERPRETATION
|    |    |    |    |    (x3)
|    |    |    |    |    {'x5': True, 'x4': True, 'x2': True, 'x1': True}
|    

#DPLL
Third sat solver.

It is actually an improvement of the backtracking algorithm where at each step, before selecting the random variable it performs a simplification of unit propagation and pure literal which are linear operations, improving the efficiency in finding a solution, if there is

In [13]:
def dpll(f, I, f_prime = None, ind = 1, ind_char = "    "):
    '''
        f: the formula to check
        I: the partial interpretation
        f_prime: a copy of the original formula during recursion for verification on partial interpretation
        ind, ind_char: for the branch indentation
    '''
    print(("|" + ind_char) * (ind-1) + "START BRANCH")
    f_check = copy.deepcopy(f) if f_prime is None else f_prime
    if copy.deepcopy(f_check).evaluate_partial(I) == True:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, I
    if copy.deepcopy(f_check).evaluate_partial(I) == False:
        print(("|" + ind_char) * (ind-1) + "NO SOLUTION FOUND!!!")
        print(("|" + ind_char) * (ind-1))
        return False, None
    print(("|" + ind_char) * ind + "PARTIAL INTERPRETATION")
    print(("|" + ind_char) * ind + "Before:\t" + str(f))
    f.partial_interpretation(I)
    print(("|" + ind_char) * ind + "After:\t" + str(f))
    print(("|" + ind_char) * ind + str(I))
    print(("|" + ind_char) * ind)
    print(("|" + ind_char) * ind + "UNIT PROPAGATION")
    f_new, I_new = UP(copy.deepcopy(f), ind, ind_char)
    print(("|" + ind_char) * ind + str(f_new))
    print(("|" + ind_char) * ind + str(I), "U", I_new)
    print(("|" + ind_char) * ind)
    for var, truth in I_new.items():
        I[var] = truth
    print(("|" + ind_char) * ind + "PURE LITERAL")
    f_new, I_new = pure_literal(copy.deepcopy(f_new), ind, ind_char)
    print(("|" + ind_char) * ind + str(f_new))
    print(("|" + ind_char) * ind + str(I), "U", I_new)
    print(("|" + ind_char) * ind)
    for var, truth in I_new.items():
        I[var] = truth

    if copy.deepcopy(f_check).evaluate_partial(I) == False:
        print(("|" + ind_char) * (ind-1) + "NO SOLUTION FOUND!!!")
        print(("|" + ind_char) * (ind-1))
        return False, None
    if copy.deepcopy(f_check).evaluate_partial(I) == True:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, I
    if len(f_new.list_clause) == 0:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, I
    for var in f.var2id.keys():
        if I.get(var) is None:
            xi = var
            break
    I_new_true = copy.deepcopy(I)
    I_new_false = copy.deepcopy(I)
    I_new_true[xi] = True
    I_new_false[xi] = False

    print(("|" + ind_char) * ind + "RANDOM VARIABLE RECURSION -", xi, "= True")
    out_b_true = dpll(copy.deepcopy(f), I_new_true, f_prime = f_check, ind = ind + 1, ind_char = ind_char)
    if out_b_true[0]:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, out_b_true[1]
    print(("|" + ind_char) * ind + "RANDOM VARIABLE RECURSION -", xi, "= False")
    out_b_false = dpll(copy.deepcopy(f), I_new_false, f_prime = f_check, ind = ind + 1, ind_char = ind_char)
    if out_b_false[0]:
        print(("|" + ind_char) * (ind-1) + "SOLUTION FOUND!!!")
        return True, out_b_false[1]

    print(("|" + ind_char) * (ind-1) + "NO SOLUTION FOUND!!!")
    print(("|" + ind_char) * (ind-1))
    return False, None

In [14]:
vars = []
vars.append([Var("x1", True), Var("x3", False), Var("x4", False)])
vars.append([Var("x2", True), Var("x6", False), Var("x4", False)])
vars.append([Var("x2", True), Var("x6", True), Var("x3", True)])
vars.append([Var("x4", True), Var("x2", True)])
vars.append([Var("x2", False), Var("x3", True), Var("x1", True)])
vars.append([Var("x2", False), Var("x6", False), Var("x3", False)])
vars.append([Var("x2", False), Var("x6", True), Var("x4", True)])
vars.append([Var("x1", False), Var("x5", False)])
vars.append([Var("x1", False), Var("x6", False)])
vars.append([Var("x6", True), Var("x3", False), Var("x5", True)])
vars.append([Var("x1", False), Var("x3", True), Var("x5", True)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)

result = dpll(f, {})
print("RESULT")
print(result)

START BRANCH
|    PARTIAL INTERPRETATION
|    Before:	(¬x1 ∨ x3 ∨ x4) ∧ (¬x2 ∨ x6 ∨ x4) ∧ (¬x2 ∨ ¬x6 ∨ ¬x3) ∧ (¬x4 ∨ ¬x2) ∧ (x2 ∨ ¬x3 ∨ ¬x1) ∧ (x2 ∨ x6 ∨ x3) ∧ (x2 ∨ ¬x6 ∨ ¬x4) ∧ (x1 ∨ x5) ∧ (x1 ∨ x6) ∧ (¬x6 ∨ x3 ∨ ¬x5) ∧ (x1 ∨ ¬x3 ∨ ¬x5)
|    After:	(¬x1 ∨ x3 ∨ x4) ∧ (¬x2 ∨ x6 ∨ x4) ∧ (¬x2 ∨ ¬x6 ∨ ¬x3) ∧ (¬x4 ∨ ¬x2) ∧ (x2 ∨ ¬x3 ∨ ¬x1) ∧ (x2 ∨ x6 ∨ x3) ∧ (x2 ∨ ¬x6 ∨ ¬x4) ∧ (x1 ∨ x5) ∧ (x1 ∨ x6) ∧ (¬x6 ∨ x3 ∨ ¬x5) ∧ (x1 ∨ ¬x3 ∨ ¬x5)
|    {}
|    
|    UNIT PROPAGATION
|    (¬x1 ∨ x3 ∨ x4) ∧ (¬x2 ∨ x6 ∨ x4) ∧ (¬x2 ∨ ¬x6 ∨ ¬x3) ∧ (¬x4 ∨ ¬x2) ∧ (x2 ∨ ¬x3 ∨ ¬x1) ∧ (x2 ∨ x6 ∨ x3) ∧ (x2 ∨ ¬x6 ∨ ¬x4) ∧ (x1 ∨ x5) ∧ (x1 ∨ x6) ∧ (¬x6 ∨ x3 ∨ ¬x5) ∧ (x1 ∨ ¬x3 ∨ ¬x5)
|    {} U {}
|    
|    PURE LITERAL
|    (¬x1 ∨ x3 ∨ x4) ∧ (¬x2 ∨ x6 ∨ x4) ∧ (¬x2 ∨ ¬x6 ∨ ¬x3) ∧ (¬x4 ∨ ¬x2) ∧ (x2 ∨ ¬x3 ∨ ¬x1) ∧ (x2 ∨ x6 ∨ x3) ∧ (x2 ∨ ¬x6 ∨ ¬x4) ∧ (x1 ∨ x5) ∧ (x1 ∨ x6) ∧ (¬x6 ∨ x3 ∨ ¬x5) ∧ (x1 ∨ ¬x3 ∨ ¬x5)
|    {} U {}
|    
|    RANDOM VARIABLE RECURSION - x6 = True
|    START BRANCH
|    |    PARTIAL INTERPRETATIO

#Backtracking and DPLL
An example of comparison between Backtracking and DPLL algorithms

In [15]:
vars = []
vars.append([Var("x1", True), Var("x2", True)])
vars.append([Var("x1", False), Var("x2", True)])
vars.append([Var("x1", True), Var("x3", True)])
vars.append([Var("x4", True), Var("x5", True)])
vars.append([Var("x4", False), Var("x5", False)])
vars.append([Var("x6", True), Var("x7", True)])
vars.append([Var("x6", False), Var("x7", False)])
clauses = [Clause(var) for var in vars]
f = Formula(clauses)

print("**************\n*BACKTRACKING*\n**************")
print(backtracking(f, {}))
print("\n\n******\n*DPLL*\n******")
print(dpll(f, {}))

**************
*BACKTRACKING*
**************
START BRANCH
|    PARTIAL INTERPRETATION
|    (¬x1 ∨ ¬x2) ∧ (x1 ∨ ¬x2) ∧ (¬x1 ∨ ¬x3) ∧ (¬x4 ∨ ¬x5) ∧ (x4 ∨ x5) ∧ (¬x6 ∨ ¬x7) ∧ (x6 ∨ x7)
|    {}
|    
|    RANDOM VARIABLE RECURSION - x6 = True
|    START BRANCH
|    |    PARTIAL INTERPRETATION
|    |    (¬x1 ∨ ¬x2) ∧ (x1 ∨ ¬x2) ∧ (¬x1 ∨ ¬x3) ∧ (¬x4 ∨ ¬x5) ∧ (x4 ∨ x5) ∧ (¬x7)
|    |    {'x6': True}
|    |    
|    |    RANDOM VARIABLE RECURSION - x5 = True
|    |    START BRANCH
|    |    |    PARTIAL INTERPRETATION
|    |    |    (¬x1 ∨ ¬x2) ∧ (x1 ∨ ¬x2) ∧ (¬x1 ∨ ¬x3) ∧ (¬x4) ∧ (¬x7)
|    |    |    {'x6': True, 'x5': True}
|    |    |    
|    |    |    RANDOM VARIABLE RECURSION - x4 = True
|    |    |    START BRANCH
|    |    |    |    PARTIAL INTERPRETATION
|    |    |    |    (¬x1 ∨ ¬x2) ∧ (x1 ∨ ¬x2) ∧ (¬x1 ∨ ¬x3) ∧ (¬x7)
|    |    |    |    {'x6': True, 'x5': True, 'x4': True}
|    |    |    |    
|    |    |    |    RANDOM VARIABLE RECURSION - x3 = True
|    |    |    |    START BRANC